In [832]:
import numpy as np
import torch
import math
import pytorchvis

In [833]:
class makeCoords(torch.nn.Module):
    def __init__(self, X):
        """
        Layer to make the coordiante pairs
        The self.coords parameter is what we care about training in the Net so we initialize it here
        and then we'll continue passing the self.coords Param (as X) through the Net
        """
        super().__init__()
        self.coords = torch.nn.Parameter(torch.tensor(torch.arange(start=0, end=len(X)*2,step=1), dtype = torch.float32))
        
    def forward(self, input):
        print("    Current Coordinates: ", [i.item() for i in self.coords])
        # We have to do ~something~ to the Param here, I think of it kind of like 'activating' the parameter
        # We can also add or subtract to 'activate' but this just returns the self.coords without any changes
        return self.coords

In [834]:
class SocialSig(torch.nn.Module):
    '''
    Class to create the social signature image
    '''
    def __init__(self, X, W):
        super(SocialSig, self).__init__()
        self.W = W
        self.outDim = [10,10]
        self.inDim = math.ceil(math.sqrt(len(X)))
        self.X = X
        

    def forward(self):
        xTemp = torch.stack([self.X, self.W])
        X = torch.sort(xTemp, dim=1, descending=False)
        buildImage = torch.reshape(X[0][0],(1, 1, self.inDim, self.inDim))
        return torch.nn.functional.interpolate(buildImage, size=([self.outDim[0], self.outDim[1]]), mode='bilinear')
       
    

        

        

In [835]:
class SocialSigNet(torch.nn.Module):
    def __init__(self, X):
        """
        In the constructor we instantiate four parameters and assign them as
        member parameters.
        """
        super().__init__()
        W = torch.nn.Parameter(torch.tensor(np.arange(0,len(X)), dtype = torch.float32, requires_grad=True))
        self.SocialSig = SocialSig(X,W)                
        self.maxPool = torch.nn.MaxPool2d(kernel_size=(10,10))  #10,10 is static here.  Will need to be dynamic based on user dim settings.

    def forward(self, X):
        out = self.SocialSig()
        out = self.maxPool(out)
        
        return out

In [836]:
# Create random tensors to hold input and outputs.
X = torch.tensor([-1.6848, -0.6, -93.2, -182.2])
y = torch.tensor([0.0359])

# Construct our model by instantiating the class defined above
model = SocialSigNet(X=X)
#print(dir(model))
print(list(model.parameters()))

[Parameter containing:
tensor([0., 1., 2., 3.], requires_grad=True)]


In [837]:
from torchviz import make_dot

make_dot(model(X), params=dict(list(model.named_parameters()))).render("backPropViz", format="png")

'backPropViz.png'

In [838]:

criterion = torch.nn.MSELoss(reduction='none')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)



for t in range(2):
    print("Epoch: ", t)
    y_pred = model(X)
    print("    Predicted Y: ", y_pred)
    loss = criterion(y_pred, y)
    print("    Loss: ", loss)

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print("\n")

print("Model.parameters")
print(model.parameters)
print("SSParam")
print(list(model.SocialSig.parameters()))
print("is_leaf")
print(list(model.SocialSig.parameters())[0].is_leaf)
print("gradfn")
print(list(model.SocialSig.parameters())[0].grad_fn)
print("Grad")
print(list(model.SocialSig.parameters())[0].grad)




Epoch:  0
    Predicted Y:  tensor([[[[-0.6000]]]], grad_fn=<MaxPool2DWithIndicesBackward>)
    Loss:  tensor([[[[0.4044]]]], grad_fn=<MseLossBackward>)


Epoch:  1
    Predicted Y:  tensor([[[[-0.6000]]]], grad_fn=<MaxPool2DWithIndicesBackward>)
    Loss:  tensor([[[[0.4044]]]], grad_fn=<MseLossBackward>)


Model.parameters
<bound method Module.parameters of SocialSigNet(
  (SocialSig): SocialSig()
  (maxPool): MaxPool2d(kernel_size=(10, 10), stride=(10, 10), padding=0, dilation=1, ceil_mode=False)
)>
SSParam
[Parameter containing:
tensor([0., 1., 2., 3.], requires_grad=True)]
reqGrad
Parameter containing:
tensor([0., 1., 2., 3.], requires_grad=True)
gradfn
None
Grad
tensor([0., 0., 0., 0.])
